In [68]:
from astropy.table import Table

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

import numpy as np
import time
from sklearn import neighbors
from sklearn import decomposition
import joblib
from vast.voidfinder._voidfinder_cython_find_next import MaskChecker
from vast.voidfinder.distance import z_to_comoving_dist
from vast.voidfinder import ra_dec_to_xyz
import pickle
import pandas as pd

Update:
- Trying to figure out if we can plot ra v dec, probs not
- If not, needed to figure out how to work with ra_dec_to_xyz() without Rgal variable (Douglass)

In [49]:
mask_file_name = "/Users/lorenzomendoza/Desktop/Research/Function/NSA_main_mask.pickle"

In [50]:
temp_infile = open(mask_file_name, "rb")
mask, mask_resolution = pickle.load(temp_infile)
temp_infile.close()

## V$^2$ void catalog

Generating galaxy array, .npy

In [51]:
V2_galzones = Table.read("/Users/lorenzomendoza/Desktop/Research/Function/V2_REVOLVER-nsa_v1_0_1_galzones.dat",format='ascii.commented_header')
V2_zonevoids = Table.read("/Users/lorenzomendoza/Desktop/Research/Function/V2_REVOLVER-nsa_v1_0_1_zonevoids.dat",format='ascii.commented_header')

V2_galzones2 = Table.read("/Users/lorenzomendoza/Desktop/Research/Function/V2_REVOLVER-nsa_v1_0_1_galzones.dat",format='ascii.commented_header')
V2_zonevoids2 = Table.read("/Users/lorenzomendoza/Desktop/Research/Function/V2_REVOLVER-nsa_v1_0_1_zonevoids.dat",format='ascii.commented_header')


# V2_gz = np.zeros(len(V2_zonevoids['zone']),dtype=int)
# V2_gz[V2_zonevoids['zone'] > -1] = 1
'''
for i in range(len(V2_gz)):
    if V2_galzones['zone'][i] > -1:
        #V2_gz[i] = V2_zonevoids['void1'][V2_galzones['zone'][i]]
        V2_gz[i] = 1
''';

In [52]:
V2_gz = np.zeros(len(V2_galzones['zone']),dtype=int)

for i in range(len(V2_gz)):
    
    if V2_zonevoids['void1'][V2_galzones['zone'][i]] > -1:
        V2_gz[i] = 1


V2_gz2 = np.zeros(len(V2_galzones2['zone']),dtype=int)

for i in range(len(V2_gz2)):
    
    if V2_zonevoids2['void1'][V2_galzones2['zone'][i]] > -1:
        V2_gz2[i] = 1

In [53]:
len(V2_gz)

194125

In [54]:
V2_gz

array([0, 0, 1, ..., 1, 0, 1])

Extract galaxy void membership. This returns the points in the data set that are V2 voids. 

If returns -1, then not V2

In [55]:
file_name = "/Users/lorenzomendoza/Desktop/Research/Function/V2_nsa_v1_0_1_gal.txt"
file_name2 = "/Users/lorenzomendoza/Desktop/Research/Function/V2_nsa_v1_0_1_gal.txt"

In [56]:
data_table_vl = Table.read(file_name, format = "ascii.commented_header")

data_table_vl2 = Table.read(file_name2, format = "ascii.commented_header")

In [57]:
omega_M = np.float32(0.3)
h = np.float32(1.0)


In [58]:
Rgal = z_to_comoving_dist(data_table_vl['redshift'].astype(np.float32),omega_M,h)
data_table_vl['Rgal'] = Rgal

Rgal2 = z_to_comoving_dist(data_table_vl2['redshift'].astype(np.float32),omega_M,h)
data_table_vl2['Rgal'] = Rgal2

In [59]:
z_boolean = data_table_vl['redshift']>0 #Edge Case: 513626 = [[-0.  0.  0.]]
data_table_vl = data_table_vl[z_boolean]


z_boolean2 = data_table_vl2['redshift']>0 #Edge Case: 513626 = [[-0.  0.  0.]]
data_table_vl2 = data_table_vl2[z_boolean]

In [60]:
galaxies_xyz = ra_dec_to_xyz(data_table_vl)
galaxies_xyz2 = ra_dec_to_xyz(data_table_vl2)

In [61]:
data_table_vl['x'] = galaxies_xyz[:,0]
data_table_vl['y'] = galaxies_xyz[:,1]
data_table_vl['z'] = galaxies_xyz[:,2]

data_table_vl2['x'] = galaxies_xyz2[:,0]
data_table_vl2['y'] = galaxies_xyz2[:,1]
data_table_vl2['z'] = galaxies_xyz2[:,2]

In [62]:
# create boolean mask
boolmask = np.isin(data_table_vl['index'], V2_galzones['gal'])

# assign values using boolean indexing
V2_galzones['x'] = data_table_vl['x'][boolmask]
V2_galzones['y'] = data_table_vl['y'][boolmask]
V2_galzones['z'] = data_table_vl['z'][boolmask]

# create boolean mask
boolmask2 = np.isin(data_table_vl['index'], V2_galzones['gal'])

# assign values using boolean indexing
V2_galzones2['x'] = data_table_vl2['x'][boolmask2]
V2_galzones2['y'] = data_table_vl2['y'][boolmask2]
V2_galzones2['z'] = data_table_vl2['z'][boolmask2]

In [71]:
x = V2_galzones2['x']
y = V2_galzones2['y']
z = V2_galzones2['z']

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

ax.scatter(x,y, z,
           color = 'maroon', s=0.1, alpha=0.5,
           label='holes')

ax.set(xlabel='X [Mpc/h]', ylabel='Y [Mpc/h]', zlabel='Z [Mpc/h]')

ax.legend(loc='upper right', fontsize=10)

plt.title("Maximal Sphere and Holes of NSA")

<IPython.core.display.Javascript object>

Text(0.5, 0.92, 'Maximal Sphere and Holes of NSA')

In [64]:
V2_galzones2

gal,zone,depth,edge,out,x,y,z
int64,int64,int64,int64,int64,float64,float64,float64
3,1036,0,1,0,-238.5497230217006,155.13200652067886,-3.771433955733498
7,1036,0,1,0,-222.8473062347432,143.68810782814072,-2.2853428144724695
10,433,3,0,0,-208.19814649187606,133.92766614907325,-0.15429539345691287
12,1036,0,1,0,-120.02429498525497,76.9669553216956,0.07190305425376976
13,1036,0,1,0,-160.641346636131,106.03878744931015,-3.216040446517236
15,1036,0,1,0,-159.8555305531387,105.75461131763109,-2.829596328341284
22,1036,0,1,0,-206.29328122879542,134.56647604765928,-2.1266625329324707
29,1036,0,1,0,-153.7215735411822,100.54651980616536,-0.6092857630969684
31,1036,0,1,0,-165.55292743628684,111.27335857977052,-2.7613607634454933


## Remove all galaxies outside the mask

For each galaxy, check to see if it is in the mask.  If it is, keep it!  If it is not, throw it away.
1. Define the variables `rmin` and `rmax`. Values are input by user. 
2. Create a boolean array called `galaxies_boolean`, with the same length as `data_table_vl`, filled with `True` values.
3. Create a `MaskChecker` object using the values `0`, `mask`, `mask_resolution`, `rmin`, and `rmax`.
4. Iterate over each element `i` in `data_table_vl`.
5. Define a variable `curr_gal` as the `i`-th element of the `galaxies_xyz` array reshaped into a 1x3 array.
6. Call the `not_in_mask` method of the `mask_checker` object on `curr_gal`, and assign the result to the variable `not_in_mask`.
7. Set the `i`-th element of the `galaxies_boolean` array to the opposite of the boolean value of `not_in_mask`.
8. After the loop has completed, print the time to run the loop.
9. Filter the `data_table_vl` array using the `galaxies_boolean` array to create a new array called `galaxies_in_mask`.
10. Print the sum of the `True` values in `galaxies_boolean`, the sum of the `False` values in `galaxies_boolean`, the shape of `galaxies_boolean`, and the `galaxies_in_mask` array.


In [17]:
def calc_volume_boundaries(void_cat_A, void_cat_B):
    """Compute the boundaries of the minimal rectangular volume (parallelpiped)
    that completely contains two void catalogs.
    
    Parameters
    ----------
    void_cat_A : astropy.Table
        Table of void data from first catalog.
    void_cat_B : astropy.Table
        Table of void data from second catalog.
        
    Returns
    -------
    x_min : float
    x_max : float
    y_min : float
    y_max : float
    z_min : float
    z_max : float
    """
    x_min = np.minimum(np.min(void_cat_A['x']), np.min(void_cat_B['x']))
    x_max = np.maximum(np.max(void_cat_A['x']), np.max(void_cat_B['x']))
    
    y_min = np.minimum(np.min(void_cat_A['y']), np.min(void_cat_B['y']))
    y_max = np.maximum(np.max(void_cat_A['y']), np.max(void_cat_B['y']))

    z_min = np.minimum(np.min(void_cat_A['z']), np.min(void_cat_B['z']))
    z_max = np.maximum(np.max(void_cat_A['z']), np.max(void_cat_B['z']))

    return x_min, x_max, y_min, y_max, z_min, z_max

def generate_grid_points(x_min, x_max, y_min, y_max, z_min, z_max):
    """Creates a dense rectangular grid of points in 3D for the void volume calculation.
    
    Returns
    -------
    xyz : list
        2D list of points in 3D space.
    """
    
    x_range = np.arange(x_min, x_max) # default spacing: 1 Megaparsec
    y_range = np.arange(y_min, y_max)
    z_range = np.arange(z_min, z_max)


    # Creating a meshgrid from the ranges to 
    X,Y,Z = np.meshgrid(x_range,y_range,z_range)

    x_points = np.ravel(X)
    y_points = np.ravel(Y)
    z_points = np.ravel(Z)
    
    point_coords = np.array([x_points, y_points, z_points])
    
    return point_coords

xmin, xmax, ymin, ymax, zmin, zmax = calc_volume_boundaries(galaxies_in_mask, galaxies_in_mask)

Calls the `calc_volume_boundaries` function on the `galaxies_in_mask` array and assigns the returned values to the variables `xmin`, `xmax`, `ymin`, `ymax`, `zmin`, and `zmax`.

---

pts = generate_grid_points(xmin, xmax, ymin, ymax, zmin, zmax)

Calls the `generate_grid_points` function with the values `xmin`, `xmax`, `ymin`, `ymax`, `zmin`, and `zmax` as arguments and assigns the returned value to the `pts` variable.

---

b = pts.shape
print(b)

Gets the shape of the `pts` array and assigns it to the `b` variable. The shape of the array is printed.


MaskChecker

In [18]:
def mask_point_filter(pts, mask, mask_resolution, rmin=0, rmax=312.89816):
    start_time = time.time()
    points_boolean = np.ones(pts.shape[1], dtype = bool)

    mask_checker = MaskChecker(0,
                            mask,
                            mask_resolution,
                            rmin,
                            rmax)

    for i in range(pts.shape[1]):
        curr_pt = pts[:,i]
        not_in_mask = mask_checker.not_in_mask(curr_pt)
        points_boolean[i] = not bool(not_in_mask)

    points_in_mask = pts[:,points_boolean]
    print('Time taken:', time.time() - start_time)
    print('Points in Mask Shape:', points_in_mask.shape)
    print('Sum of Points IN:', np.sum(points_boolean))
    print('Sum of Points OUT:', np.sum(~points_boolean))
    print('Boolean Shape:', points_boolean.shape)
    print('Points in Mask:',points_in_mask)
    return points_in_mask, points_boolean


In [19]:
def kd_tree(void_cat):
    """We are creating a function to make a KDTree to find the number of points in 
    and out of a catalogue.
    
    Parameters
    ----------
    point_coords: ndarray has a shape of (3,N)
        This is the list of points to query the given void catalogue. N is the number of points given. 
    void_cat: Astropy Table
        This is the given void catalogue.
    
    Returns
    -------
    true_inside: ndarray of shape (N,1)
        Is this the boolean array of length N (same length as point_coords). True means that 1 point 
        is inside the hole.
    """
#############
    cx = void_cat['x']
    cy = void_cat['y']
    cz = void_cat['z']

    sphere_coords = np.array([cx, cy, cz])

    #The .T is meant to transpose the array from (3,1054) to (1054,3)
    sphere_tree = neighbors.KDTree(sphere_coords.T)
    # print("KDTree")

##############
    
    return sphere_tree

In [20]:
def point_query(point_coords, sphere_tree, void_cat):
    # print("Starting Query")
    #Void cat classifcation
    true_inside = np.zeros(point_coords.shape[1])

    idx = sphere_tree.query(point_coords.T, k = 1, return_distance=False)
    
    #true_inside = void_cat[idx]
    for i in range(len(idx)):
        true_inside[i] = void_cat[idx[i]]
    
    return true_inside

In [21]:
def count_points(points_in_mask, galzones_V1, galzones_V2, V2_gz, V2_gz2):
    start_time = time.time()
    (var, n_points) = points_in_mask.shape

    # Takes about 1.5 mins per query
    points_in_mask_copy = points_in_mask.copy()

    kdTree_V1 = kd_tree(galzones_V1)
    kdTree_V2 = kd_tree(galzones_V2)

    true_inside_V1 = point_query(points_in_mask_copy, kdTree_V1, V2_gz)
    count_in_V1 = np.sum(true_inside_V1)
    count_out_V1 = n_points - count_in_V1

    true_inside_V2 = point_query(points_in_mask_copy, kdTree_V2, V2_gz2)
    count_in_V2 = np.sum(true_inside_V2)
    count_out_V2 = n_points - count_in_V2

    inside_both = np.sum(np.logical_and(true_inside_V1, true_inside_V2))
    inside_neither = np.sum(np.logical_not(np.logical_or(true_inside_V1, true_inside_V2)))
    inside_V1 = np.sum(np.logical_and(true_inside_V1, np.logical_not(true_inside_V2)))
    inside_V2 = np.sum(np.logical_and(true_inside_V2, np.logical_not(true_inside_V1)))
    



    print("Runtime:", time.time() - start_time)
    print('\nNumber of points inside V1:', count_in_V1)
    print('\nNumber of points outside V2:', count_out_V1)
    print('\nNumber of points inside V1:', count_in_V2)
    print('\nNumber of points outside V2:', count_out_V2)
    print("\nThis is the total number of points: {}".format(n_points))
    # print("\nThis is the total number of points in Delaunay: {}".format(total_DEL))
    return (count_in_V1, count_out_V1, count_in_V2, count_out_V2, inside_both, inside_neither, inside_V1, inside_V2, n_points)

In [22]:
xmin, xmax, ymin, ymax, zmin, zmax = calc_volume_boundaries(V2_galzones, V2_galzones2)

#This line makes creates the points in between 



In [23]:
pts = generate_grid_points(xmin, xmax, ymin, ymax, zmin, zmax)

b = pts.shape
print(b)

(3, 64749960)


In [24]:
points_in_mask, points_boolean = mask_point_filter(pts, mask, mask_resolution)


Time taken: 93.98937821388245
Points in Mask Shape: (3, 23119361)
Sum of Points IN: 23119361
Sum of Points OUT: 41630599
Boolean Shape: (64749960,)
Points in Mask: [[-108.17809884 -108.17809884 -108.17809884 ... -138.17809884
  -138.17809884 -138.17809884]
 [-293.30349342 -293.30349342 -293.30349342 ...  270.69650658
   270.69650658  270.69650658]
 [  -5.14470225   -4.14470225   -3.14470225 ...   71.85529775
    72.85529775   73.85529775]]


In [25]:
(count_in_V1, count_out_V1, count_in_V2, count_out_V2, inside_both, inside_neither, inside_V1, inside_V2, n_points) = count_points(points_in_mask, V2_galzones, V2_galzones2, V2_gz, V2_gz2)

Runtime: 334.66118931770325

Number of points inside V1: 20857709.0

Number of points outside V2: 2261652.0

Number of points inside V1: 20857709.0

Number of points outside V2: 2261652.0

This is the total number of points: 23119361


In [26]:
r_V1 = count_in_V1 / n_points
r_V2 = count_in_V2 / n_points
r_V1_V2 = np.sum(inside_both) / n_points
r_not_V1_V2 = np.sum(inside_neither) / n_points
r_V1_not_V2 = np.sum(inside_V1) / n_points
r_V2_not_V1 = np.sum(inside_V2) / n_points

average_V1 = np.mean(count_in_V1)
r_average_V1 = average_V1 / n_points
std_V1 = np.std(count_in_V1)
r_std_V1 = std_V1 / n_points
print("\nNumber of points inside V1: {}".format(count_in_V1))
print("\nNumber of points outside V1: {}".format(count_out_V1))
print("\nStandard Deviation of V1: {}".format(std_V1))

print('\nRatio of  V1 Points:', r_average_V1)
print('\nRatio SD of V1:', r_std_V1)
      
average_V2 = np.mean(count_in_V2)
r_average_V2 = average_V2 / n_points
std_V2 = np.std(count_in_V2)
r_std_V2 = std_V2 / n_points
print("\nNumber of points inside V2: {}".format(count_in_V2))
print("\nNumber of points outside V2: {}".format(count_out_V2))
print("\nStandard Deviation of V2: {}".format(std_V2))
print('\nRatio of V2 Points:', r_average_V2)
print('\nRatio SD of V2:', r_std_V2)


average_inside = np.mean(inside_both)
r_average_inside = average_inside / n_points

std_both = np.std(inside_both)
r_std_both = std_both / n_points
print("\nNumber of points inside both: {}".format(inside_both))
print("\nNumber of points outside both: {}".format(inside_neither))
print("\nStandard Deviation of both: {}".format(std_both))
print('\nRatio of Points Inside:', r_average_inside)
print('\nRatio SD of Both:', r_std_both)


average_outside = np.mean(inside_neither)
r_average_outside = average_outside / n_points

std_outside = np.std(inside_neither)
r_std_outside = std_outside / n_points
print("\nNumber of points inside neither: {}".format(inside_neither))
print("\nNumber of points outside neither: {}".format(inside_neither))
print('\nRatio of Points Outside:', r_average_outside)
print('\nRatio SD of Neither:', r_std_outside)


# average_in_V1 = np.mean(inside_V1)
# r_average_in_V1 = average_in_V1 / n_points

# std_in_V1 = np.std(inside_neither)
# r_std_in_V1 = std_in_V1 / n_points


# print('\nRatio of Points in V1:', r_average_in_V1)
# print('\nRatio SD:', r_std_in_V1)


# average_in_V2 = np.mean(inside_V2)
# r_average_in_V2 = average_in_V2 / n_points

# std_in_V2 = np.std(inside_neither)
# r_std_in_V2 = std_in_V2 / n_points

# print('\nRatio of Points in V2:', average_in_V2)
# print('\nRatio SD:', r_std_in_V2)


Number of points inside V1: 20857709.0

Number of points outside V1: 2261652.0

Standard Deviation of V1: 0.0

Ratio of  V1 Points: 0.9021749779329974

Ratio SD of V1: 0.0

Number of points inside V2: 20857709.0

Number of points outside V2: 2261652.0

Standard Deviation of V2: 0.0

Ratio of V2 Points: 0.9021749779329974

Ratio SD of V2: 0.0

Number of points inside both: 20857709

Number of points outside both: 2261652

Standard Deviation of both: 0.0

Ratio of Points Inside: 0.9021749779329974

Ratio SD of Both: 0.0

Number of points inside neither: 2261652

Number of points outside neither: 2261652

Ratio of Points Outside: 0.09782502206700264

Ratio SD of Neither: 0.0


In [27]:
average_V2 = np.mean(count_in_V2)
r_average_V2 = average_V2 / n_points
std_V2 = np.std(count_in_V2)
r_std_V2 = std_V2 / n_points
print("\nNumber of points inside V2: {}".format(count_in_V2))
print("\nNumber of points outside V2: {}".format(count_out_V2))
print("\nStandard Deviation of V2: {}".format(std_V2))
print('\nRatio of V2 Points:', r_average_V2)
print('\nRatio SD:', r_std_V2)





Number of points inside V2: 20857709.0

Number of points outside V2: 2261652.0

Standard Deviation of V2: 0.0

Ratio of V2 Points: 0.9021749779329974

Ratio SD: 0.0


In [28]:
average_inside = np.mean(inside_both)
r_average_inside = average_inside / n_points

std_both = np.std(inside_both)
r_std_both = std_both / n_points
print("\nNumber of points inside both: {}".format(inside_both))
print("\nNumber of points outside both: {}".format(inside_neither))
print("\nStandard Deviation of both: {}".format(std_both))
print('\nRatio of Points Inside:', r_average_inside)
print('\nRatio SD:', r_std_both)





Number of points inside both: 20857709

Number of points outside both: 2261652

Standard Deviation of both: 0.0

Ratio of Points Inside: 0.9021749779329974

Ratio SD: 0.0


In [29]:
average_outside = np.mean(inside_neither)
r_average_outside = average_outside / n_points

std_outside = np.std(inside_neither)
r_std_outside = std_outside / n_points
print("\nNumber of points inside neither: {}".format(inside_neither))
print("\nNumber of points outside neither: {}".format(inside_neither))
print("\nStandard Deviation of neither: {}".format(std_outside))
print('\nRatio of Points Outside:', r_average_outside)
print('\nRatio SD:', r_std_outside)


Number of points inside neither: 2261652

Number of points outside neither: 2261652

Standard Deviation of neither: 0.0

Ratio of Points Outside: 0.09782502206700264

Ratio SD: 0.0


In [30]:
import pandas as pd
import numpy as np

def calculate_ratios_and_stats(count_in_V1, count_out_V1, count_in_V2, count_out_V2, inside_both, inside_neither, inside_V1, inside_V2, n_points):
    r_V1 = count_in_V1 / n_points
    r_V2 = count_in_V2 / n_points
    r_V1_V2 = np.sum(inside_both) / n_points
    r_not_V1_V2 = np.sum(inside_neither) / n_points
    r_V1_not_V2 = np.sum(inside_V1) / n_points
    r_V2_not_V1 = np.sum(inside_V2) / n_points

    average_V1 = np.mean(count_in_V1)
    r_average_V1 = average_V1 / n_points
    std_V1 = np.std(count_in_V1)
    r_std_V1 = std_V1 / n_points
    
    average_V2 = np.mean(count_in_V2)
    r_average_V2 = average_V2 / n_points
    std_V2 = np.std(count_in_V2)
    r_std_V2 = std_V2 / n_points
    
    average_inside = np.mean(inside_both)
    r_average_inside = average_inside / n_points
    std_both = np.std(inside_both)
    r_std_both = std_both / n_points
    
    average_outside = np.mean(inside_neither)
    r_average_outside = average_outside / n_points
    std_outside = np.std(inside_neither)
    r_std_outside = std_outside / n_points
    
    results = pd.DataFrame({
        'Category': ['V1', 'V2', 'Both', 'Neither', 'V1 not V2', 'V2 not V1'],
        'Number of points': [count_in_V1, count_in_V2, np.sum(inside_both), np.sum(inside_neither), np.sum(inside_V1), np.sum(inside_V2)],
        'Number of points outside': [count_out_V1, count_out_V2, np.nan, np.nan, np.nan, np.nan],
        'Ratio of points': [r_V1, r_V2, r_V1_V2, r_not_V1_V2, r_V1_not_V2, r_V2_not_V1],
        'Average number of points': [average_V1, average_V2, average_inside, average_outside, np.nan, np.nan],
        'Standard deviation of points': [std_V1, std_V2, std_both, std_outside, np.nan, np.nan],
        'Ratio of average number of points': [r_average_V1, r_average_V2, r_average_inside, r_average_outside, np.nan, np.nan],
        'Ratio of standard deviation': [r_std_V1, r_std_V2, r_std_both, r_std_outside, np.nan, np.nan]
    })
    
    return results.set_index('Category')


In [31]:
results = calculate_ratios_and_stats(count_in_V1, count_out_V1, count_in_V2, count_out_V2, inside_both, inside_neither, inside_V1, inside_V2, n_points)


In [32]:
results

,Number of points,Number of points outside,Ratio of points,Average number of points,Standard deviation of points,Ratio of average number of points,Ratio of standard deviation
Category,,,,,,,
V1,20857709.0,2261652.0,0.902175,20857709.0,0.0,0.902175,0.0
V2,20857709.0,2261652.0,0.902175,20857709.0,0.0,0.902175,0.0
Both,20857709.0,NaN,0.902175,20857709.0,0.0,0.902175,0.0
Neither,2261652.0,NaN,0.097825,2261652.0,0.0,0.097825,0.0
V1 not V2,0.0,NaN,0.000000,NaN,NaN,NaN,NaN
V2 not V1,0.0,NaN,0.000000,NaN,NaN,NaN,NaN


In [33]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_catalogues(pts, points_boolean, inside_V1, inside_V2):
    
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(projection='3d')
    
    ax.scatter(pts[0, ~points_boolean], pts[1, ~points_boolean], pts[2, ~points_boolean],
               color='grey', s=10, alpha=0.5)
    
    ax.scatter(pts[0, points_boolean & inside_V1], pts[1, points_boolean & inside_V1], pts[2, points_boolean & inside_V1],
               color='red', s=10, alpha=0.5, label='in catalogue V1')
    
    ax.scatter(pts[0, points_boolean & inside_V2], pts[1, points_boolean & inside_V2], pts[2, points_boolean & inside_V2],
               color='orange', s=10, alpha=0.5, label='in catalogue V2')
    
    ax.set(xlabel='X [Mpc/h]', ylabel='Y [Mpc/h]', zlabel='Z [Mpc/h]')
    ax.legend()
    
    plt.title("Comparison of Catalogues")
    
    plt.show()


In [75]:
def plot_points(points_in_mask, pts):
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(projection='3d')

    ax.scatter(points_in_mask[0, ::100], points_in_mask[1, ::100], points_in_mask[2, ::100],
               color='red', s=10, alpha=1,
               label="Points in Mask")
    
    ax.scatter(pts[0, ::100], pts[1, ::100], pts[2, ::100],
               color='blue', s=10, alpha=0.1,
               label="Points from Void Volume")
    
    ax.set(xlabel='X [Mpc/h]',
           ylabel='Y [Mpc/h]',
           zlabel='Z [Mpc/h]')
    
    plt.legend()
    fig.savefig('myplot.png')
    plt.show()

In [76]:
plot = plot_points(points_in_mask, pts)

<IPython.core.display.Javascript object>

In [72]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(projection='3d')
##
# This will take the beautiful 3D array of pts and plot along each axis
# (x, y, z)
##
ax.scatter(points_in_mask[0,::100],points_in_mask[1,::100], points_in_mask[2,::100],
           color = 'red', s=10, alpha=0.5,
          )
ax.set(xlabel='X [Mpc/h]',
       ylabel='Y [Mpc/h]',
       zlabel='Z [Mpc/h]')

plt.title("Points in Mask");

<IPython.core.display.Javascript object>

In [38]:
points_in_mask

array([[-108.17809884, -108.17809884, -108.17809884, ..., -138.17809884,
        -138.17809884, -138.17809884],
       [-293.30349342, -293.30349342, -293.30349342, ...,  270.69650658,
         270.69650658,  270.69650658],
       [  -5.14470225,   -4.14470225,   -3.14470225, ...,   71.85529775,
          72.85529775,   73.85529775]])

In [37]:
plot_catalogues(pts, points_boolean, inside_V1, inside_V2)

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [42]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

ax.scatter(data_table_vl['x'],data_table_vl['y'], data_table_vl['z'],
           color = 'maroon', s=0.1, alpha=0.5,
           label='holes')
ax.set(xlabel='X [Mpc/h]', ylabel='Y [Mpc/h]', zlabel='Z [Mpc/h]')

ax.legend(loc='upper right', fontsize=10)

plt.title("Maximal Sphere and Holes of NSA");

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
x

-159.47584420504052
-296.1574894141973
-138.89347406859915
-238.5497230217006
-308.4810128885206
-160.14597434128163
-219.2735589331227
-222.8473062347432
-327.9392589096939
-15.805483229243551
-208.19814649187606
